# <span style='color:#0D2BA5'>Projet 6 - Classifiez automatiquement des biens de consommation : API</span>

## <span style='background:#33B5FF'>Table des Matieres</span>

In [23]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## <span style='background:#33B5FF'>1. Introduction</span>

**Mission**
*****

Aprés avoir démontré la faisabilité de regrouper automatiquement des produits de même catégorie ! 
Maintenant, on me propose de passer à la deuxième itération. Mettre en place une data augmentation afin d’optimiser le modèle, car le client souhaiterais élargir sa gamme de produits, en particulier dans l’épicerie fine. 
*****
**Source**
*****
Le jeux de donnée est disponible à l'adresse : [Données](https://rapidapi.com/edamam/api/edamam-food-and-grocery-database)
*****



**Axes d'orientations**
*****

Nous allons :

- Tester la collecte de produits à base de “champagne” via l’API.
- Fournir une extraction des 10 premiers produits dans un fichier “.csv”, contenant pour chaque produit les données suivantes : foodId, label, category, foodContentsLabel, image.



## <span style='background:#33B5FF'>2. Bibliothèques d'analyse Python</span>

In [1]:
import pandas as pd
import json
from pandas import json_normalize

## <span style='background:#33B5FF'>3. Extraction des données de l'API</span>

In [22]:
import requests

url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

querystring = {"health[0]":"alcohol-free","category[0]":"generic-foods","nutrition-type":"cooking","ingr":"champagne"}

headers = {
	"X-RapidAPI-Key": "1f69451b14msh6e5d93e009c1b4cp17a666jsn1066e3d51151",
	"X-RapidAPI-Host": "edamam-food-and-grocery-database.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


In [14]:
dictr = response.json()

In [15]:
dictr

{'text': 'champagne',
 'parsed': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'}}],
 'hints': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'},
   'measures': [{'uri': '

In [16]:
[
    (food['food']['category'], food['food']['label'])
    for food in dictr['hints']
]

[('Generic foods', 'Champagne'),
 ('Packaged foods', 'Champagne Vinaigrette, Champagne'),
 ('Packaged foods', 'Champagne Vinaigrette, Champagne'),
 ('Packaged foods', 'Champagne Vinaigrette, Champagne'),
 ('Packaged foods', 'Champagne Vinaigrette, Champagne'),
 ('Packaged foods', 'Champagne Dressing, Champagne'),
 ('Generic meals', 'Champagne Buttercream'),
 ('Generic meals', 'Champagne Sorbet'),
 ('Generic meals', 'Champagne Truffles'),
 ('Generic meals', 'Champagne Vinaigrette'),
 ('Generic meals', 'Champagne Chicken'),
 ('Generic meals', 'Strawberry Champagne'),
 ('Generic meals', 'Champagne Jelly'),
 ('Generic meals', 'Champagne Punch'),
 ('Generic meals', 'Champagne Sangria'),
 ('Packaged foods', 'Champagne Cotton Candy, Champagne'),
 ('Packaged foods', 'Champagne Simply Dressed Vinaigrette, Champagne'),
 ('Generic meals', 'Champagne Cake'),
 ('Generic meals', 'Champagne Cupcakes'),
 ('Packaged foods', 'Champagne Vinegar')]

In [17]:
FIELDS = [
    "foodId",
    "label",
    "category",
    "foodContentsLabel",
    "image",
]

def get_food(data):
    food = data['food']
    return [ food.get(f) for f in FIELDS ]

def get_foods(foods):
    return pd.DataFrame(
        [
            get_food(parsed)
            for parsed in foods
        ],
        columns=FIELDS,
    )

In [18]:
all_foods = None
foods = get_foods(dictr.get('hints') or [])
if all_foods is None:
    all_foods = foods
else:
    all_foods = pd.concat([all_foods, foods])
#response = requests.get(next['href']).json()

all_foods

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,None,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,None
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,None
4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,None
5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,None
7,food_byap67hab6evc3a0f9w1oag3s0qf,Champagne Sorbet,Generic meals,Sugar; Lemon juice; brandy; Champagne; Peach,None
8,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,None
9,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,None


## <span style='background:#33B5FF'>4. Enregistrer les données en .CSV</span>

In [19]:
all_foods.head(10).to_csv('champagne_test_api.csv')

In [20]:
df = pd.read_csv('champagne_test_api.csv')
df.head(10)

,Unnamed: 0,foodId,label,category,foodContentsLabel,image
0,0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
4,4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
5,5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
7,7,food_byap67hab6evc3a0f9w1oag3s0qf,Champagne Sorbet,Generic meals,Sugar; Lemon juice; brandy; Champagne; Peach,NaN
8,8,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
9,9,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN
